In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("preprocessed-datasets/CongressionVoting_prepro.csv")

df['class'] = df['class'].map({'democrat': 0, 'republican': 1})

# Convert to numpy array
data = df.to_numpy()

# Split the data into training (80%) and testing (20%) sets
np.random.shuffle(data)
split = int(0.8 * len(data))
train_set = data[:split]
test_set = data[split:]

# Split training set to X and y
X_train = train_set[:, :-1]
y_train = train_set[:, -1]

# Split testing set to X and y
X_test = test_set[:, :-1]
y_test = test_set[:, -1]

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(18)
    W_h = np.random.rand(input_size, hidden_size)
    W_o = np.random.rand(hidden_size, output_size)
    return W_h, W_o

def forward_propagation(X, W_h, W_o):
    hidden_layer_input = np.dot(X, W_h)
    hidden_layer_output = sigmoid(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_output, W_o)
    output_layer_output = sigmoid(output_layer_input)

    return hidden_layer_output, output_layer_output

def backward_propagation(X, y, hidden_layer_output, output_layer_output, W_o):
    output_error = y - output_layer_output
    output_delta = output_error * sigmoid_derivative(output_layer_output)

    hidden_layer_error = output_delta.dot(W_o.T)
    hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer_output)

    return hidden_layer_delta, output_delta

def update_weights(X, hidden_layer_output, output_delta, hidden_layer_delta, W_h, W_o, learning_rate):
    W_o += hidden_layer_output.T.dot(output_delta) * learning_rate
    W_h += X.T.dot(hidden_layer_delta) * learning_rate

def train_neural_network(X_train, y_train, epochs=10, hidden_size=25, learning_rate=0.01):
    input_size = X_train.shape[1]
    output_size = 1

    W_h, W_o = initialize_parameters(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        hidden_layer_output, output_layer_output = forward_propagation(X_train, W_h, W_o)

        hidden_layer_delta, output_delta = backward_propagation(X_train, y_train.reshape(-1, 1), hidden_layer_output, output_layer_output, W_o)

        update_weights(X_train, hidden_layer_output, output_delta, hidden_layer_delta, W_h, W_o, learning_rate)

        loss = np.mean(np.square(y_train.reshape(-1, 1) - output_layer_output))
        print(f"Epoch {epoch}, Loss: {loss}")

    return W_h, W_o

def predict(X, W_h, W_o):
    _, output_layer_output = forward_propagation(X, W_h, W_o)
    return (output_layer_output > 0.5).astype(int).flatten()

# Train the neural network
W_h, W_o = train_neural_network(X_train, y_train, epochs=10, hidden_size=int(2/3 * X_train.shape[1] + 2), learning_rate=0.01)

# Test the neural network
y_pred = predict(X_test, W_h, W_o)

# Evaluate the accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

Epoch 0, Loss: 0.628460900082247
Epoch 1, Loss: 0.6211840564094468
Epoch 2, Loss: 0.6088586955668208
Epoch 3, Loss: 0.5849475313989462
Epoch 4, Loss: 0.528782219014993
Epoch 5, Loss: 0.37629548139575797
Epoch 6, Loss: 0.22663658257687114
Epoch 7, Loss: 0.22648893719666813
Epoch 8, Loss: 0.22644437710357937
Epoch 9, Loss: 0.22583611844567084
Accuracy: 0.5454545454545454
